In [1]:
import os
import sys
from datetime import datetime
from IPython.display import HTML

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

WIDTH = 1100
pd.options.display.float_format = 'R$ {:,.2f}'.format

caminho_commons = os.path.join('../..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('../..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('../..')
sys.path.append(caminho_virasana)

from ajna_commons.flask.conf import DATABASE, MONGODB_URI

from bhadrasana.models import engine


def AnoMes(row):
    return str(row.Ano) + str(row['Mês']).zfill(2)


KeyboardInterrupt: 

# Colocar os códigos embaixo

In [ ]:
#  4 é fase arquivada sem resultado, eliminar!!!
SQL_OUTLET = '''SELECT r.nome as Recinto, ficha.id as 'Ficha', rvf.id as 'RVF', rvf.datahora 'Data de Abertura',
     ficha.numeroCEmercante, rvf.numerolote as Contêiner, ficha.fase
     FROM ovr_ovrs ficha
     INNER JOIN ovr_verificacoesfisicas rvf ON  rvf.ovr_id = ficha.id
     inner join ovr_flags_ovr flags on flags.rvf_id = ficha.id
     INNER JOIN ovr_recintos r ON r.id = ficha.recinto_id
     where flags.flag_id = 9
     ORDER BY r.nome, rvf.datahora'''

df_pendente_outlet = pd.read_sql(SQL_OUTLET, engine)
today_ = datetime.today()
def calcula_dias(x):
    try:
        return int((today_ - x['Data de Abertura']).days)
    except:
        return 0
            

df_pendente_outlet['Dias sem conclusão'] = \
    df_pendente_outlet.apply(calcula_dias, axis=1)
lista_ovr = ', '.join([str(i) for i in df_pendente_outlet.Ficha.values])

SQL_EVENTOS = \
    '''SELECT ovr_id as Ficha, tipoevento_id, t.nome, e.create_date, motivo FROM ovr_eventos e
    inner join ovr_tiposevento t on e.tipoevento_id = t.id  where ovr_id in (%s)
    ORDER BY ovr_id;'''

df_eventos = pd.read_sql(SQL_EVENTOS % lista_ovr, engine)


# Tipos de Evento : 36 Em análise da Fiscalização
#                    7 Aguardando quantificação do Recinto
#                   21 Termo de Guarda informado
#                   8 Recebimento de quantificação do Recinto
df_pendente_outlet = df_pendente_outlet.drop_duplicates(subset=['Ficha'])
df_pendente_outlet = df_pendente_outlet.set_index('Ficha')
df_pendente_outlet = df_pendente_outlet.join(
    df_eventos[df_eventos.tipoevento_id == 36].drop_duplicates(
        ('Ficha', 'tipoevento_id'))[['Ficha', 'motivo']].set_index('Ficha')
)

In [ ]:
df_pendente_outlet = df_pendente_outlet.rename(columns={'motivo': 'Parecer'}).fillna('--')

In [ ]:
df_pendente_outlet.replace({'SANTOS/BRASIL SA OPERADOR PORTUARIO': 'SANTOS BRASIL PARTICIPACOES LTDA' }, inplace=True)

In [ ]:
tipos_eventos = {36: 'Análise',
                 7: 'Saneamento solicitado',
                 8: 'Saneamento recebido',
                 21: 'TG recebido'}
for tipoevento_id, descricao in tipos_eventos.items():
    df_pendente_outlet = df_pendente_outlet.join(
        df_eventos[df_eventos.tipoevento_id == tipoevento_id].drop_duplicates(
            ('Ficha', 'tipoevento_id'))[['Ficha', 'create_date']].set_index('Ficha')
    )
    df_pendente_outlet = df_pendente_outlet.rename(columns={'create_date': descricao}).fillna('Sem evento')

In [ ]:
recintos = df_pendente_outlet['Recinto'].unique()
df_pendente_outlet['Recinto'].value_counts()

In [ ]:
def duracao(x):
    try:
        return (x['Saneamento recebido'] - x['Data de Abertura']).days
    
    except:
        try:
            return (x['TG recebido'] - x['Data de Abertura']).days
        except:
            return 0
df_pendente_outlet['Duração'] = df_pendente_outlet.apply(duracao, axis=1)

In [ ]:
len(df_pendente_outlet[df_pendente_outlet['Recinto'] == 'BANDEIRANTES DEICMAR - CLIA ALEMOA'])

In [ ]:
def mostra_recinto(recinto, filtrar_concluidos=False):
    print(recinto, len(df_pendente_outlet[df_pendente_outlet['Recinto'] == recinto]))
    df_pendentes = df_pendente_outlet.reset_index()
    df_pendentes = df_pendentes[['Ficha', 'Recinto', 'Data de Abertura', 'numeroCEmercante', 'Dias sem conclusão',
                                 'Parecer', 'Contêiner', 'Saneamento recebido', 'TG recebido', 'fase', 'Duração']]
    df_pendentes.sort_values(['Ficha'], inplace=True)
    if filtrar_concluidos:
        df_pendentes=df_pendentes[df_pendentes['fase'] != 3]
        df_pendentes=df_pendentes[df_pendentes['fase'] != 4]
        df_pendentes=df_pendentes[df_pendentes['TG recebido'] == 'Sem evento']
    return df_pendentes[df_pendentes['Recinto'] == recinto]

In [ ]:
mostra_recinto(recintos[0])

In [ ]:
mostra_recinto(recintos[0], True).drop(columns=['Parecer', 'fase']).style.hide_index()

In [ ]:
mostra_recinto(recintos[1])

In [ ]:
mostra_recinto(recintos[1], True).drop(columns=['Parecer', 'fase']).style.hide_index()

In [ ]:
mostra_recinto(recintos[7], True).drop(columns=['Parecer', 'fase']).style.hide_index()

In [ ]:
paradigma = mostra_recinto(recintos[4])
paradigma

In [ ]:
paradigma[paradigma['Duração']==0]

In [ ]:
paradigma_conclusos = paradigma[paradigma['Duração']!=0]
print('Duração média: ', paradigma_conclusos['Duração'].mean())
print('Quantidade concluídos:', len(paradigma_conclusos))
print('Data primeiro e último:', min(paradigma_conclusos['Data de Abertura']), max(paradigma_conclusos['Data de Abertura']))

In [ ]:
resumo_recintos = {}
for recinto in recintos:
    df_recinto = df_pendente_outlet[(df_pendente_outlet.Recinto == recinto) & (df_pendente_outlet.fase !=4)]
    concluidos = 0
    pendentes = 0
    tempoc = 0
    tempop = 0
    for row in df_recinto.iterrows():
        if row[1]['Duração'] == 0:  # Não concluído / pendente
            p = row[1]['Dias sem conclusão']
            tempop += p
            pendentes += 1
        else:  # Concluído
            c = row[1]['Duração']
            tempoc += c
            concluidos += 1
    resumo_recintos[recinto] = {
        'concluídos': concluidos,
        'duração média': 0 if concluidos == 0 else int(tempoc / concluidos),
        'pendentes': pendentes,
        'dias sem conclusão média': 0 if pendentes == 0 else int(tempop / pendentes)
    }
df_resumo = pd.DataFrame.from_dict(resumo_recintos, orient='index')
df_resumo['% Concluído'] = (df_resumo['concluídos'] / (df_resumo['concluídos'] + df_resumo['pendentes']))*100


In [ ]:
pd.options.display.float_format = '{:,.2f}%'.format
df_resumo.dropna()